In [ ]:
import pandas as pd
import GladUtil as gd

## 수수료 지급율 변경전후 비교

In [119]:
## glad표준 엑셀 파일
# path ='C:/Users/User/Desktop/myga/'
path = 'E:/글로벌백업(180329)/03.수수료&시상/glad수수료규정/원수사지급율/'
in_file1 = '수수료(2021.8)선지급형_생보.xlsx'
in_file2 = '수수료(2021.6)선지급형_생보.xlsx'
outfile = 'e:/temp/생보수수료변경_2108.xlsx'
c = '회사 보종 상품명 납기 환산율 환산율2 환산율3 1회 수입P1 2회 3회 4회 5회 6회 7회 8회 9회 10회 11회 12회 지급계1y 수입p1y 13회 14회 15회 16회 17회 18회 19회 20회 21회 22회 23회 24회 지급계2y 수입P2y 25회 26회 27회 28회 29회 30회 31회 32회 33회 34회 35회 36회 지급계3y 수입P3y 지급계 수입P 비고'
columns = c.split(' ')

def pre_proc(df):
    df['회사'].fillna(method='ffill',inplace=True)
    df['보종'] = df.보종.replace(0,None)
    df['보종'].fillna(method='ffill',inplace=True)
    df.loc[:,'보종'] = df['보종'].str.replace('\n','').str.replace(' ','')
    df.loc[:,'회사'] = df['회사'].str.replace('\n','').str.replace(' ','')
    df.fillna(0,inplace=True )
    df = df[df['지급계']!=0]
    return df


sheet = '생명보험(회사)'
df = pd.read_excel(path+in_file1,sheet_name=sheet,header=4,names=columns,usecols="a:ba")
df_aft = pre_proc(df)

sheet = '생명보험(회사)'
df = pd.read_excel(path+in_file2,sheet_name=sheet,header=4,names=columns,usecols="a:ba")
df_bef = pre_proc(df)

In [135]:
df_join = df_aft.join(df_bef.set_index(['회사','보종','납기'])[['환산율','환산율2','환산율3','1회','지급계2y','지급계3y','지급계']], 
                      on=(['회사','보종','납기']), rsuffix='_변경전')
# df_bef.set_index(['회사','보종','납기'])
df_join['총지급율변동'] = df_join['지급계'] - df_join['지급계_변경전']
df_join['초회지급율변동'] = df_join['1회'] - df_join['1회_변경전']
df_join['2차년지급율변동'] = df_join['지급계2y'] - df_join['지급계2y_변경전']
df_join['3차년지급율변동'] = df_join['지급계3y'] - df_join['지급계3y_변경전']
df_join.fillna(0,inplace=True)

# df_changed = df_join[ (abs(df_join['총지급율변동']) > 0.05) | (abs(df_join['초회지급율변동']) > 0.05)]
exp = f"초회지급율변동 > 0.05 or 총지급율변동 > 0.05 or 지급계_변경전==0"
df_changed = df_join.query(exp)
changed = df_changed.iloc[:,4:] * 100   # %를 숫자로
# df_changed.iloc[:, 4:] = changed      # waring나와 loc로 변경
df_changed.loc[:, df_changed.columns[4:]] = changed

df_changed.to_excel(outfile)

In [132]:
# -------------------------------------

In [ ]:
df_aft['구분']='변경후'
df_bef['구분']='변경전'
# df_outer = pd.concat([df_aft,df_bef]).sort_values(['회사','보종','구분'])
df_concat = pd.concat([df_bef.set_index(['회사','보종','납기']),df_aft.set_index(['회사','보종','납기'])],join='inner')

In [ ]:
df_concat

In [ ]:
df=pd.DataFrame({"a":[10,20,30],"b":[50,130,450],"c":["a","bb","cc"]})
dfa=pd.DataFrame({"a":[10,20,30],"b":[51,131,451],"d":["a","bb","cc"],"c":["ad","bdb","dcc"]}, index=[1,2,3])
# cols=["a","b"]
# df[cols].diff()
pd.concat([df,dfa],join='inner', axis=1)

In [ ]:
df = pd.merge(df_bef, df_aft, how='inner', on=['회사','보종','납기'])
df.columns

In [ ]:
from functools import reduce
dfs = [df_bef, df_aft]
cols = [*df_bef.columns]
reduce(lambda left,right: pd.merge(left,right,on=cols), dfs)

In [ ]:
list(df_bef.columns)